In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mnist-dataset/train-images.idx3-ubyte
/kaggle/input/mnist-dataset/t10k-labels.idx1-ubyte
/kaggle/input/mnist-dataset/t10k-images.idx3-ubyte
/kaggle/input/mnist-dataset/train-labels.idx1-ubyte
/kaggle/input/mnist-dataset/t10k-labels-idx1-ubyte/t10k-labels-idx1-ubyte
/kaggle/input/mnist-dataset/t10k-images-idx3-ubyte/t10k-images-idx3-ubyte
/kaggle/input/mnist-dataset/train-labels-idx1-ubyte/train-labels-idx1-ubyte
/kaggle/input/mnist-dataset/train-images-idx3-ubyte/train-images-idx3-ubyte


In [18]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import numpy as np
import struct
from PIL import Image

In [19]:
# Custom dataset class for MNIST
class CustomMNISTDataset(Dataset):
    def __init__(self, images_path, labels_path, transform=None):
        self.images_path = images_path
        self.labels_path = labels_path
        self.transform = transform
        self.images, self.labels = self.load_data()

    def load_data(self):
        # Load images
        with open(self.images_path, 'rb') as img_file:
            _, num_images, rows, cols = struct.unpack(">IIII", img_file.read(16))
            images = np.fromfile(img_file, dtype=np.uint8).reshape(num_images, 1, rows, cols)

        # Load labels
        with open(self.labels_path, 'rb') as lbl_file:
            _, num_labels = struct.unpack(">II", lbl_file.read(8))
            labels = np.fromfile(lbl_file, dtype=np.uint8)

        return images, labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        # Convert the image to a PIL image
        image = Image.fromarray(image[0], mode='L')

        if self.transform:
            image = self.transform(image)

        return image, label



In [20]:
# Set up data transformation
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalize the image
])


In [21]:
# Set paths to the MNIST files in the Kaggle environment
train_images_path = '/kaggle/input/mnist-dataset/train-images.idx3-ubyte'
train_labels_path = '/kaggle/input/mnist-dataset/train-labels.idx1-ubyte'
test_images_path = '/kaggle/input/mnist-dataset/t10k-images.idx3-ubyte'
test_labels_path = '/kaggle/input/mnist-dataset/t10k-labels.idx1-ubyte'


In [22]:
# Create datasets
train_dataset = CustomMNISTDataset(train_images_path, train_labels_path, transform=transform)
test_dataset = CustomMNISTDataset(test_images_path, test_labels_path, transform=transform)


In [23]:
# Create DataLoader instances for batching and shuffling
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [24]:

# Check dataset size
print(f"Training dataset size: {len(train_dataset)}")
print(f"Testing dataset size: {len(test_dataset)}")

Training dataset size: 60000
Testing dataset size: 10000


In [25]:

# Example of accessing one batch
data_iter = iter(train_loader)
images, labels = next(data_iter)
print(f"Image batch shape: {images.shape}, Label batch shape: {labels.shape}")

Image batch shape: torch.Size([64, 1, 28, 28]), Label batch shape: torch.Size([64])


In [35]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        # Convolutional Layer 1
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)  # 28x28 -> 28x28
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)  # 28x28 -> 14x14
        # Convolutional Layer 2
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)  # 14x14 -> 14x14
        # Convolutional Layer 3
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)  # 14x14 -> 14x14
        # Fully Connected Layers
        self.fc1 = None  # Placeholder, will initialize later

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # Apply Conv1 + Pooling
        x = self.pool(F.relu(self.conv2(x)))  # Apply Conv2 + Pooling
        x = self.pool(F.relu(self.conv3(x)))  # Apply Conv3 + Pooling

        # Calculate the flattened size dynamically
        batch_size = x.size(0)
        x = x.view(batch_size, -1)  # Flatten the output for fully connected layer
        
        # Initialize the fully connected layers after calculating the size
        if self.fc1 is None:
            # Dynamically calculate the flattened size based on the input
            self.fc1 = nn.Linear(x.size(1), 512)  # Input size should match the flattened size
            self.fc2 = nn.Linear(512, 10)  # Output layer for 10 classes (digits 0-9)

        x = F.relu(self.fc1(x))  # Fully connected layer 1
        x = self.fc2(x)  # Fully connected layer 2 (output layer)
        return x


In [36]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [37]:
# Instantiate the model and move it to the GPU if available
model = CNNModel().to(device)

In [38]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [39]:
# Function to train the model
def train(model, train_loader, criterion, optimizer, device, num_epochs=5):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")



In [40]:
# Function to evaluate the model
def evaluate(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy}%")
    return accuracy



In [ ]:
# Train the model
train(model, train_loader, criterion, optimizer, device, num_epochs=5)

In [41]:

# Evaluate the model
evaluate(model, test_loader, device)

Test Accuracy: 14.47%


14.47